In [7]:
!pip install findspark

You should consider upgrading via the 'C:\Users\looka\OneDrive\Documents\pySpark\spark_env\Scripts\python.exe -m pip install --upgrade pip' command.


In [9]:
!pip install pyspark

Using legacy 'setup.py install' for pyspark, since package 'wheel' is not installed.
    Running setup.py install for pyspark: started
    Running setup.py install for pyspark: still running...
    Running setup.py install for pyspark: finished with status 'done'


You should consider upgrading via the 'C:\Users\looka\OneDrive\Documents\pySpark\spark_env\Scripts\python.exe -m pip install --upgrade pip' command.


In [11]:
!pip install plotly

You should consider upgrading via the 'C:\Users\looka\OneDrive\Documents\pySpark\spark_env\Scripts\python.exe -m pip install --upgrade pip' command.


In [13]:
!pip install matplotlib

  Using cached matplotlib-3.9.0-cp39-cp39-win_amd64.whl (7.9 MB)
  Using cached pyparsing-3.1.2-py3-none-any.whl (103 kB)
  Using cached kiwisolver-1.4.5-cp39-cp39-win_amd64.whl (56 kB)
  Using cached contourpy-1.2.1-cp39-cp39-win_amd64.whl (182 kB)
  Using cached numpy-2.0.0-cp39-cp39-win_amd64.whl (16.5 MB)
  Using cached importlib_resources-6.4.0-py3-none-any.whl (38 kB)
  Using cached fonttools-4.53.0-cp39-cp39-win_amd64.whl (2.2 MB)
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)


You should consider upgrading via the 'C:\Users\looka\OneDrive\Documents\pySpark\spark_env\Scripts\python.exe -m pip install --upgrade pip' command.


In [35]:
!pip install pandas

  Using cached pandas-2.2.2-cp39-cp39-win_amd64.whl (11.6 MB)
  Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)


You should consider upgrading via the 'C:\Users\looka\OneDrive\Documents\pySpark\spark_env\Scripts\python.exe -m pip install --upgrade pip' command.


In [1]:
import findspark
from functools import reduce
from pyspark import SparkContext
from pyspark.sql import SparkSession, Window, Row
from pyspark.sql.functions import *
from pyspark.sql.types import *
import plotly as py
import matplotlib.pyplot as plt

In [2]:
# entry point - creating basic spark session using SparkSession.builder 
spark = SparkSession.builder.appName("erstespark").getOrCreate()
        

In [3]:
## function for loading the dataframe
# https://spark.apache.org/docs/latest/sql-data-sources-generic-options.html
def load_dataframe(f):
    df = spark.read.format('csv').options(header = 'true').load(f)
    return df

In [4]:
## creating a dataframe
matches = load_dataframe('./data/Matches.csv')
matches.limit(10).show()

+--------+---+------+----------+--------------+--------------+----+----+---+
|Match_ID|Div|Season|      Date|      HomeTeam|      AwayTeam|FTHG|FTAG|FTR|
+--------+---+------+----------+--------------+--------------+----+----+---+
|       1| D2|  2009|2010-04-04|    Oberhausen|Kaiserslautern|   2|   1|  H|
|       2| D2|  2009|2009-11-01|   Munich 1860|Kaiserslautern|   0|   1|  A|
|       3| D2|  2009|2009-10-04| Frankfurt FSV|Kaiserslautern|   1|   1|  D|
|       4| D2|  2009|2010-02-21| Frankfurt FSV|     Karlsruhe|   2|   1|  H|
|       5| D2|  2009|2009-12-06|         Ahlen|     Karlsruhe|   1|   3|  A|
|       6| D2|  2009|2010-04-03|  Union Berlin|     Karlsruhe|   1|   1|  D|
|       7| D2|  2009|2009-08-14|     Paderborn|     Karlsruhe|   2|   0|  H|
|       8| D2|  2009|2010-03-08|     Bielefeld|     Karlsruhe|   0|   1|  A|
|       9| D2|  2009|2009-09-26|Kaiserslautern|     Karlsruhe|   2|   0|  H|
|      10| D2|  2009|2009-11-21| Hansa Rostock|     Karlsruhe|   2|   1|  H|

In [5]:
## 
matches.columns

['Match_ID',
 'Div',
 'Season',
 'Date',
 'HomeTeam',
 'AwayTeam',
 'FTHG',
 'FTAG',
 'FTR']

In [6]:
alt_columns = matches.columns[-3:]
neue_columns = ["Home_Team_Goal", "Away_Team_Goal", "Final_Result"]

In [7]:
alt_neue_columns = [*zip(alt_columns, neue_columns)]

In [8]:
alt_neue_columns

[('FTHG', 'Home_Team_Goal'),
 ('FTAG', 'Away_Team_Goal'),
 ('FTR', 'Final_Result')]

In [9]:
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.withColumnRenamed.html
for alt_columns, neue_columns in alt_neue_columns:
    matches = matches.withColumnRenamed(alt_columns, neue_columns)

In [10]:
matches.columns

['Match_ID',
 'Div',
 'Season',
 'Date',
 'HomeTeam',
 'AwayTeam',
 'Home_Team_Goal',
 'Away_Team_Goal',
 'Final_Result']

In [11]:
matches.limit(5).toPandas()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,Home_Team_Goal,Away_Team_Goal,Final_Result
0,1,D2,2009,2010-04-04,Oberhausen,Kaiserslautern,2,1,H
1,2,D2,2009,2009-11-01,Munich 1860,Kaiserslautern,0,1,A
2,3,D2,2009,2009-10-04,Frankfurt FSV,Kaiserslautern,1,1,D
3,4,D2,2009,2010-02-21,Frankfurt FSV,Karlsruhe,2,1,H
4,5,D2,2009,2009-12-06,Ahlen,Karlsruhe,1,3,A


#### Frage 1: Who are the winners of the D1 division in the Germany Football Association (Bundesliga) in the 2010 to 2016?

In [105]:
## filter division with D1, last decade (2010 - 2019)
## https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.filter.html
bundesliga = matches.filter((col('Season') >= 2010) & (col('Season') <= 2016) & (col('Div') == 'D1'))
# bundesliga.limit(5).toPandas()

## indicate the winner loser and drow in simple way
## https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.withColumn.html
bundesliga = bundesliga.withColumn('HomeTeamWin', when(col('Final_Result') == 'H', 1).otherwise(0)) \
                        .withColumn('AwayTeamWin', when(col('Final_Result') == 'A', 1).otherwise(0)) \
                        .withColumn('Drow', when(col('Final_Result') == 'D', 1).otherwise(0))

               



In [106]:
#bundesliga.limit(5).toPandas()
bundesliga.columns

['Match_ID',
 'Div',
 'Season',
 'Date',
 'HomeTeam',
 'AwayTeam',
 'Home_Team_Goal',
 'Away_Team_Goal',
 'Final_Result',
 'HomeTeamWin',
 'AwayTeamWin',
 'Drow']

In [107]:
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.GroupedData.agg.html
home = bundesliga.groupby('Season', 'HomeTeam') \
                .agg(sum('HomeTeamWin').alias('TotalHomeWin'),
                     sum('AwayTeamWin').alias('TotalHomeLoss'),
                     sum('Drow').alias('TotalHomeDrow'),
                     sum('Home_Team_Goal').alias('Home_Scored_Goals'),
                     sum('Away_Team_Goal').alias('Home_Against_Goals')) \
                .withColumnRenamed('HomeTeam', 'Team') 

home.toPandas()

,Season,Team,TotalHomeWin,TotalHomeLoss,TotalHomeDrow,Home_Scored_Goals,Home_Against_Goals
0,2011,Hamburg,3,7,7,19.0,29.0
1,2015,Hertha,9,3,5,24.0,15.0
2,2012,Hannover,9,3,5,34.0,23.0
3,2014,Paderborn,4,7,6,21.0,31.0
4,2012,M'gladbach,8,4,5,27.0,20.0
...,...,...,...,...,...,...,...
121,2012,Fortuna Dusseldorf,5,6,6,24.0,28.0
122,2014,Dortmund,9,5,3,26.0,15.0
123,2015,Hamburg,5,8,4,20.0,23.0
124,2016,Bayern Munich,13,0,4,55.0,9.0


In [108]:
away = bundesliga.groupby('Season', 'AwayTeam') \
                .agg(sum('AwayTeamWin').alias('TotalAwayWin'),
                     sum('HomeTeamWin').alias('TotalAwayLoss'),
                     sum('Drow').alias('TotalAwayDrow'),
                     sum('Away_Team_Goal').alias('Away_Scored_Goals'),
                     sum('Home_Team_Goal').alias('Away_Against_Goals')) \
                .withColumnRenamed('AwayTeam', 'Team') 

In [109]:
# Werder Bremen	
bundesliga.filter((col('HomeTeam') == 'Werder Bremen') | (col('AwayTeam') == 'Werder Bremen')).show()

+--------+---+------+----------+-------------+--------------+--------------+--------------+------------+-----------+-----------+----+
|Match_ID|Div|Season|      Date|     HomeTeam|      AwayTeam|Home_Team_Goal|Away_Team_Goal|Final_Result|HomeTeamWin|AwayTeamWin|Drow|
+--------+---+------+----------+-------------+--------------+--------------+--------------+------------+-----------+-----------+----+
|     489| D1|  2010|2011-01-29|Werder Bremen| Bayern Munich|             1|             3|           A|          0|          1|   0|
|     498| D1|  2010|2011-05-07|Werder Bremen|      Dortmund|             2|             0|           H|          1|          0|   0|
|     542| D1|  2010|2010-11-13|Werder Bremen| Ein Frankfurt|             0|             0|           D|          0|          0|   1|
|     560| D1|  2010|2010-08-28|Werder Bremen|       FC Koln|             4|             2|           H|          1|          0|   0|
|     576| D1|  2010|2010-10-16|Werder Bremen|      Freiburg| 

In [110]:
away.toPandas()

,Season,Team,TotalAwayWin,TotalAwayLoss,TotalAwayDrow,Away_Scored_Goals,Away_Against_Goals
0,2011,Hamburg,5,7,5,16.0,28.0
1,2015,Hertha,5,9,3,18.0,27.0
2,2012,Hannover,4,12,1,26.0,39.0
3,2014,Paderborn,3,10,4,10.0,34.0
4,2012,M'gladbach,4,7,6,18.0,29.0
...,...,...,...,...,...,...,...
121,2012,Fortuna Dusseldorf,2,12,3,15.0,29.0
122,2014,Dortmund,4,9,4,21.0,27.0
123,2015,Hamburg,6,7,4,20.0,23.0
124,2016,Bayern Munich,12,2,3,34.0,13.0


In [111]:
print(away.columns)
print(home.columns)

['Season', 'Team', 'TotalAwayWin', 'TotalAwayLoss', 'TotalAwayDrow', 'Away_Scored_Goals', 'Away_Against_Goals']
['Season', 'Team', 'TotalHomeWin', 'TotalHomeLoss', 'TotalHomeDrow', 'Home_Scored_Goals', 'Home_Against_Goals']


In [132]:
## inner join :https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/api/pyspark.pandas.DataFrame.join.html?highlight=inner%20join
## window : https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Window.orderBy.html
window = ['Season']
window = Window.partitionBy(window).orderBy(col('WinPercentage').desc(), col('GoalDifference').desc())
table = home.join(away, ['Team', 'Season'], how ='inner') \
        .withColumn('GoalsScored', col('Home_Scored_Goals') + col('Away_Scored_Goals')) \
        .withColumn('GoalsAgainst', col('Home_Against_Goals') + col('Away_Against_Goals')) \
        .withColumn('GoalDifference', col('GoalsScored') - col('GoalsAgainst')) \
        .withColumn('Win', col('TotalHomeWin') + col('TotalAwayWin')) \
        .withColumn('Loss', col('TotalHomeLoss') + col('TotalAwayLoss')) \
        .withColumn('Drow', col('TotalHomeDrow') + col('TotalAwayDrow')) \
        .withColumn('WinPercentage', round((100 * col('Win') /(col('Win') + col('Loss') + col('Drow'))), 2))\
        .drop('Away_Scored_Goals', 'Away_Against_Goals', 'Home_Scored_Goals', 'Home_Against_Goals')\
        .drop('TotalAwayWin', 'TotalHomeWin', 'TotalHomeLoss', 'TotalAwayLoss', 'TotalAwayDrow', 'TotalHomeDrow')\
        .withColumn('TeamPosition', rank().over(window))



In [133]:
table.toPandas()

,Team,Season,GoalsScored,GoalsAgainst,GoalDifference,Win,Loss,Drow,WinPercentage,TeamPosition
0,Dortmund,2010,67.0,22.0,45.0,23,5,6,67.65,1
1,Leverkusen,2010,64.0,44.0,20.0,20,6,8,58.82,2
2,Bayern Munich,2010,81.0,40.0,41.0,19,7,8,55.88,3
3,Hannover,2010,49.0,45.0,4.0,19,12,3,55.88,4
4,Mainz,2010,52.0,39.0,13.0,18,12,4,52.94,5
...,...,...,...,...,...,...,...,...,...,...
121,Wolfsburg,2016,34.0,52.0,-18.0,10,17,7,29.41,14
122,Hamburg,2016,33.0,61.0,-28.0,10,16,8,29.41,15
123,Augsburg,2016,35.0,51.0,-16.0,9,14,11,26.47,16
124,Ingolstadt,2016,36.0,57.0,-21.0,8,18,8,23.53,17


In [131]:
table = table.filter(col('TeamPosition') == 1).orderBy(asc('Season'))
table.toPandas()

,Team,Season,GoalsScored,GoalsAgainst,GoalDifference,Win,Loss,Drow,WinPercentage,TeamPosition
0,Dortmund,2010,67.0,22.0,45.0,23,5,6,67.65,1
1,Dortmund,2011,80.0,25.0,55.0,25,3,6,73.53,1
2,Bayern Munich,2012,98.0,18.0,80.0,29,1,4,85.29,1
3,Bayern Munich,2013,94.0,23.0,71.0,29,2,3,85.29,1
4,Bayern Munich,2014,80.0,18.0,62.0,25,5,4,73.53,1
5,Bayern Munich,2015,80.0,17.0,63.0,28,2,4,82.35,1
6,Bayern Munich,2016,89.0,22.0,67.0,25,2,7,73.53,1


In [121]:
table.groupby('Team').agg(count('Team').alias("TotalChampionShips"),
                        avg('WinPercentage').alias('AvgWinPct'),
                         avg('GoalDifference').alias('AvgGoalDifference'),
                         avg('Win').alias('AvgWin'),
                         avg('Loss').alias('AvgLoss'),
                         avg('Drow').alias('AvgDrow'))\
                    .orderBy(desc("TotalChampionShips")).toPandas().round(1)

,Team,TotalChampionShips,AvgWinPct,AvgGoalDifference,AvgWin,AvgLoss,AvgDrow
0,Bayern Munich,5,80.0,68.6,27.2,2.4,4.4
1,Dortmund,2,70.6,50.0,24.0,4.0,6.0


#### Frage 2: Which teams have been relegated in that 6 years?

In [125]:
teams = load_dataframe('./data/Teams.csv')

In [126]:
teams.limit(5).toPandas()

,Season,TeamName,KaderHome,AvgAgeHome,ForeignPlayersHome,OverallMarketValueHome,AvgMarketValueHome,StadiumCapacity
0,2017,Bayern Munich,27,26,15,597950000,22150000,75000
1,2017,Dortmund,33,25,18,416730000,12630000,81359
2,2017,Leverkusen,31,24,15,222600000,7180000,30210
3,2017,RB Leipzig,30,23,15,180130000,6000000,42959
4,2017,Schalke 04,29,24,17,179550000,6190000,62271


In [134]:
table.toPandas()

,Team,Season,GoalsScored,GoalsAgainst,GoalDifference,Win,Loss,Drow,WinPercentage,TeamPosition
0,Dortmund,2010,67.0,22.0,45.0,23,5,6,67.65,1
1,Leverkusen,2010,64.0,44.0,20.0,20,6,8,58.82,2
2,Bayern Munich,2010,81.0,40.0,41.0,19,7,8,55.88,3
3,Hannover,2010,49.0,45.0,4.0,19,12,3,55.88,4
4,Mainz,2010,52.0,39.0,13.0,18,12,4,52.94,5
...,...,...,...,...,...,...,...,...,...,...
121,Wolfsburg,2016,34.0,52.0,-18.0,10,17,7,29.41,14
122,Hamburg,2016,33.0,61.0,-28.0,10,16,8,29.41,15
123,Augsburg,2016,35.0,51.0,-16.0,9,14,11,26.47,16
124,Ingolstadt,2016,36.0,57.0,-21.0,8,18,8,23.53,17


In [138]:
## relegated last three teams
relegated = table.filter((col('TeamPosition') == 16) | (col('TeamPosition') == 17) | (col('TeamPosition') == 18)).orderBy(asc('Season'))

In [140]:
## teams of 2010
relegated.filter(col('Season') == 2010).toPandas()

,Team,Season,GoalsScored,GoalsAgainst,GoalDifference,Win,Loss,Drow,WinPercentage,TeamPosition
0,Wolfsburg,2010,43.0,48.0,-5.0,9,14,11,26.47,16
1,Ein Frankfurt,2010,31.0,49.0,-18.0,9,18,7,26.47,17
2,St Pauli,2010,35.0,68.0,-33.0,8,21,5,23.53,18


#### Frage 3: Does Octoberfest affect the performance of Bundesliga? 

In [141]:
matches.toPandas()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,Home_Team_Goal,Away_Team_Goal,Final_Result
0,1,D2,2009,2010-04-04,Oberhausen,Kaiserslautern,2,1,H
1,2,D2,2009,2009-11-01,Munich 1860,Kaiserslautern,0,1,A
2,3,D2,2009,2009-10-04,Frankfurt FSV,Kaiserslautern,1,1,D
3,4,D2,2009,2010-02-21,Frankfurt FSV,Karlsruhe,2,1,H
4,5,D2,2009,2009-12-06,Ahlen,Karlsruhe,1,3,A
...,...,...,...,...,...,...,...,...,...
24620,46770,E0,2016,2017-05-21,Liverpool,Middlesbrough,3,0,H
24621,46771,E0,2016,2017-05-21,Man United,Crystal Palace,2,0,H
24622,46772,E0,2016,2017-05-21,Southampton,Stoke,0,1,A
24623,46773,E0,2016,2017-05-21,Swansea,West Brom,2,1,H


In [ ]:
octo_fest = matches.filter(col('Div') == 'D1').filter(col)

#### Frage 4: Which season of Bundesliga was the most competitive in the last decade?

#### Frage 5: What's the best month to watch Bundesliga?